In [208]:
## import packages
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

## hide warnings
import warnings
warnings.filterwarnings('ignore')

In [209]:
## load the csv data into data frame
csv_PATH = "/Users/yaodongqiu/Desktop/Data4Policy/Geospatial Data Science Internship/Datasets/telangana_fires.csv"
telangana_fires_orginal_df = pd.read_csv (csv_PATH)

In [293]:
telangana_fires = telangana_fires_orginal_df[["latitude","longitude","acq_date"]]
telangana_fires["geometry"] = gpd.points_from_xy(telangana_fires["longitude"], telangana_fires["latitude"])
telangana_fires["year"] = telangana_fires.acq_date.str[0:4]
telangana_fires = gpd.GeoDataFrame(telangana_fires[["year","geometry"]])
telangana_fires

,year,geometry
0,2015,POINT (80.53140 18.37770)
1,2015,POINT (80.98930 17.58980)
2,2015,POINT (81.16120 17.60280)
3,2015,POINT (80.31350 17.52410)
4,2015,POINT (80.34040 17.75710)
...,...,...
18780,2021,POINT (77.82570 18.39230)
18781,2021,POINT (77.76110 18.49430)
18782,2021,POINT (79.30120 18.48050)
18783,2021,POINT (80.78150 17.95060)


In [294]:
## load district label data
PATH_shape = "/Users/yaodongqiu/Desktop/gadm40_IND_shp/gadm40_IND_3.shp"
india_shapefile = gpd.read_file(PATH_shape)
Telangana_orginal = india_shapefile[india_shapefile["NAME_1"] == "Telangana"]
Telangana = Telangana_orginal[["NAME_3","geometry"]]
Telangana

,NAME_3,geometry
1954,Adilabad,"POLYGON ((78.84434 19.75900, 78.84563 19.75835..."
1955,Asifabad,"POLYGON ((78.96471 19.57028, 78.95446 19.56272..."
1956,Boath,"POLYGON ((78.19715 19.33173, 78.18712 19.33861..."
1957,Chinnur,"POLYGON ((79.92231 19.15713, 79.90597 19.14179..."
1958,Khanapur,"POLYGON ((78.51988 19.22164, 78.52913 19.23259..."
...,...,...
2021,Jangaon,"POLYGON ((79.08975 18.02090, 79.09614 18.01483..."
2022,Mulug,"POLYGON ((80.33668 18.59052, 80.33879 18.58821..."
2023,n.a. ( 1711),"POLYGON ((80.15270 17.38762, 80.15408 17.38008..."
2024,Narsampet,"POLYGON ((80.30102 17.96696, 80.29185 17.96178..."


In [341]:
join_df = telangana_fires.sjoin(Telangana)
join_df = join_df.rename(columns={"NAME_3": "adm_name"})
join_df = join_df[["adm_name","year"]]
join_df.groupby(["adm_name","year"],as_index=False).value_counts()

,adm_name,year,count
0,Achampet,2015,138
1,Achampet,2016,84
2,Achampet,2017,32
3,Achampet,2018,150
4,Achampet,2019,127
...,...,...,...
491,n.a. ( 1728),2017,3
492,n.a. ( 1728),2018,9
493,n.a. ( 1728),2019,3
494,n.a. ( 1728),2020,8
